In [2]:
import os

import numpy as np
from sklearn.decomposition import PCA

from utils import load_hidden_representations_from_hdf5, read_templates_from_file

----

In [3]:
# params
log_dir = "/logfiles"
model = "bigscience-T0_3B" # bigscience-T0_B or bigscience-T0
module = "encoder" # encoder
# task = "rte"
task = "cb"
# task = "wic"

In [4]:
assert module == "encoder" # TODO(mm): support decoder as well

## Prompts

In [5]:
df = read_templates_from_file(f"/t0-analysis/prompts/{task}.csv")
display(df)

,name,template,category,includes_targets,targets,shuffle
0,null_pattern,{premise} {hypothesis},neutral,False,none,False
1,null_pattern_reversed,{hypothesis} {premise},neutral,False,none,False
2,gpt_3_true_false_neither,"{premise} Question: {hypothesis} True, False, ...",instructive,True,"True, False, Neither",False
3,gpt_3_yes_no_maybe,"{premise} Question: {hypothesis} Yes, No, or M...",instructive,True,"Yes, No, Maybe",False
4,mnli_crowdsource,{premise} Using only the above description and...,instructive,True,"Correct, Incorrect, Inconclusive",False
5,always_sometimes_never,"Suppose it's true that {premise} Then, is ""{hy...",instructive,True,"Always, Sometimes, Never",False
6,based_on_previous_passage,"{premise} Based on the previous passage, is it...",instructive,True,"Yes, No, Maybe",False
7,infer,"Suppose {premise} Can we infer that ""{hypothes...",instructive,True,"Yes, No, Maybe",False
8,claim,"{premise} Based on that information, is the cl...",instructive,True,"True, False, Inconclusive",False
9,consider,"{premise} Keeping in mind the above text, cons...",instructive,True,"Always, Sometimes, Never",False


## PCA

In [6]:
def unison_shuffled_copies(a, b):
    # from: https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [7]:
# RTE patterns
# use_pattern = [
#     "null_pattern",
#     "null_pattern_reversed",
#     "gpt_3_yes_no",
#     "gpt_3_yes_no_shuffled",
#     "gpt_3_true_false",
#     "gpt_3_true_false_shuffled",
#     "start_with_the",
#     "mnli_crowdsource",
#     "based_on_previous_passage",
#     "infer",
#     "follow",
#     "imply",
#     "guaranteed",
#     "justified", 
#     "must_be_true",
#     "should_assume"
# ]

# CB patterns
use_pattern = [
    "null_pattern",
    "null_pattern_reversed",
    "gpt_3_true_false_neither",
    "gpt_3_yes_no_maybe",
    "mnli_crowdsource",
    "always_sometimes_never",
    "based_on_previous_passage",
    "infer",
    "claim",
    "consider",
    "follow",
    "imply",
    "guaranteed",
    "guaranteed_possible",
    "justified",
    "must_be_true",
    "should_assume",
    "take_the_following",
]

# WIC patterns
# use_pattern = [
#     "gpt_3",
#     "gpt_3_yes_no",
#     "affirmation",
#     "grammar_homework",
#     "polysemous",
#     "question_context",
#     "question_meaning",
#     "question_meaning_yes_no",
#     "same_sense",
#     "similar_sense",
#     "similar_sense_yes_no",
# ]

In [8]:
# for layer in range(0, 10):
for layer in range(0, 25):
# for layer in range(24, 25):
    print('layer=', layer)
    file_names, prompt_names = [], []

    for _, row in df.iterrows():
        if row['name'] in use_pattern:
            file_names.append(f"{task}/{model}/{module}/{row['name']}/hidden_represenations_layer{layer}_avg.hdf5",)
            prompt_names.append(row['name'])


    # load hidden representations from hdf5 file
    representations = None
    classes = []
    n_sequences = 0

    for idx, file_name in enumerate(file_names):
        hidden_representations = load_hidden_representations_from_hdf5(os.path.join(log_dir, file_name))
        # print(hidden_representations.shape)
        n_sequences = hidden_representations.shape[0]

        if representations is None:
            representations = hidden_representations
        else:
            representations = np.concatenate((representations, hidden_representations), axis=0)

        classes += n_sequences * [idx] # assign representations to classes
    
    classes = np.asarray(classes)

    # shuffle representations and classes
    X, y = unison_shuffled_copies(representations, classes)
    print(X.shape, y.shape)

    # perform PCA on hidden representations
    print('PCA for prompts:', prompt_names)

    for n_components in range(1, 4):
        pca = PCA(n_components=n_components)
        pca.fit(X)

        # variance explained by each of the principal components
        print(f"model:{model}; module:{module}; layer:{layer}; n_components: {n_components}; variance explained: {pca.explained_variance_ratio_}")
        print(np.sum(pca.explained_variance_ratio_))
    print('\n')


layer= 0


Reading embeddings: 100%|██████████| 56/56 [00:00<00:00, 4999.06it/s]


(1008, 2048) (1008,)
PCA for prompts: ['null_pattern', 'null_pattern_reversed', 'gpt_3_true_false_neither', 'gpt_3_yes_no_maybe', 'mnli_crowdsource', 'always_sometimes_never', 'based_on_previous_passage', 'infer', 'claim', 'consider', 'follow', 'imply', 'guaranteed', 'guaranteed_possible', 'justified', 'must_be_true', 'should_assume', 'take_the_following']
model:bigscience-T0_3B; module:encoder; layer:0; n_components: 1; variance explained: [0.9334196]
0.9334196
model:bigscience-T0_3B; module:encoder; layer:0; n_components: 2; variance explained: [0.9334196  0.01033202]
0.94375163
model:bigscience-T0_3B; module:encoder; layer:0; n_components: 3; variance explained: [0.93341976 0.01033203 0.00871127]
0.9524631


layer= 1


Reading embeddings: 100%|██████████| 56/56 [00:00<00:00, 5601.07it/s]


(1008, 2048) (1008,)
PCA for prompts: ['null_pattern', 'null_pattern_reversed', 'gpt_3_true_false_neither', 'gpt_3_yes_no_maybe', 'mnli_crowdsource', 'always_sometimes_never', 'based_on_previous_passage', 'infer', 'claim', 'consider', 'follow', 'imply', 'guaranteed', 'guaranteed_possible', 'justified', 'must_be_true', 'should_assume', 'take_the_following']
model:bigscience-T0_3B; module:encoder; layer:1; n_components: 1; variance explained: [0.26871872]
0.26871872
model:bigscience-T0_3B; module:encoder; layer:1; n_components: 2; variance explained: [0.26871872 0.20745684]
0.47617555
model:bigscience-T0_3B; module:encoder; layer:1; n_components: 3; variance explained: [0.26871875 0.20745681 0.0925732 ]
0.5687487


layer= 2


...

(1008, 2048) (1008,)
PCA for prompts: ['null_pattern', 'null_pattern_reversed', 'gpt_3_true_false_neither', 'gpt_3_yes_no_maybe', 'mnli_crowdsource', 'always_sometimes_never', 'based_on_previous_passage', 'infer', 'claim', 'consider', 'follow', 'imply', 'guaranteed', 'guaranteed_possible', 'justified', 'must_be_true', 'should_assume', 'take_the_following']
model:bigscience-T0_3B; module:encoder; layer:22; n_components: 1; variance explained: [0.9805164]
0.9805164
model:bigscience-T0_3B; module:encoder; layer:22; n_components: 2; variance explained: [0.9805164  0.00273535]
0.98325175
model:bigscience-T0_3B; module:encoder; layer:22; n_components: 3; variance explained: [0.98051655 0.00273535 0.0016682 ]
0.98492014


layer= 23


Reading embeddings: 100%|██████████| 56/56 [00:00<00:00, 4002.68it/s]


(1008, 2048) (1008,)
PCA for prompts: ['null_pattern', 'null_pattern_reversed', 'gpt_3_true_false_neither', 'gpt_3_yes_no_maybe', 'mnli_crowdsource', 'always_sometimes_never', 'based_on_previous_passage', 'infer', 'claim', 'consider', 'follow', 'imply', 'guaranteed', 'guaranteed_possible', 'justified', 'must_be_true', 'should_assume', 'take_the_following']
model:bigscience-T0_3B; module:encoder; layer:23; n_components: 1; variance explained: [0.97186935]
0.97186935
model:bigscience-T0_3B; module:encoder; layer:23; n_components: 2; variance explained: [0.97186935 0.00372159]
0.97559094
model:bigscience-T0_3B; module:encoder; layer:23; n_components: 3; variance explained: [0.97186935 0.00372159 0.00223501]
0.97782594


layer= 24


Reading embeddings: 100%|██████████| 56/56 [00:00<00:00, 5695.19it/s]


(1008, 2048) (1008,)
PCA for prompts: ['null_pattern', 'null_pattern_reversed', 'gpt_3_true_false_neither', 'gpt_3_yes_no_maybe', 'mnli_crowdsource', 'always_sometimes_never', 'based_on_previous_passage', 'infer', 'claim', 'consider', 'follow', 'imply', 'guaranteed', 'guaranteed_possible', 'justified', 'must_be_true', 'should_assume', 'take_the_following']
model:bigscience-T0_3B; module:encoder; layer:24; n_components: 1; variance explained: [0.10334048]
0.103340484
model:bigscience-T0_3B; module:encoder; layer:24; n_components: 2; variance explained: [0.10334042 0.08637156]
0.18971199
model:bigscience-T0_3B; module:encoder; layer:24; n_components: 3; variance explained: [0.10334035 0.08637142 0.06249465]
0.25220644


